## Usar ▶▶ (Restart the kernel and run all celdas) para ejecución completa.

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from helpers import reader, raw_details, data_cleaner_aggregator, writer
spark = SparkSession.builder.appName("App").getOrCreate()

In [ ]:
# Usar unicamente para declarar las variables de entorno  -  sustituyendo las rutas correspondientes para JAVA y HADOOP
# import os
# os.environ["PATH"] += r";C:\hadoop-3.3.6\bin"
# os.environ["JAVA_HOME"] = "C:\\Program Files\\Eclipse Adoptium\\jdk-21.0.7.6-hotspot"

# Lectura principal del insumo RAW

In [ ]:
path = "raw/data_prueba_tecnica.csv"  # Delimitar ruta path donde esta ubicado el archivo raw.
reader_type = "csv" # Tipo de lectura
rd = reader.Reader(spark, reader_type, path)
df = rd.reader_process()


# Detalles generales insumo RAW

In [ ]:
campos_llave = ["id"] # Pasar uno o más columnas que conformen la llave primaria 
rwd = raw_details.RawDetails(df,campos_llave)
rwd.crude_stage_details()

# Etapa de limpieza, generación de información (ventas por día)

In [ ]:
dca = data_cleaner_aggregator.DataCleanerAggregator(df) # Recibe el dataframe raw
df_detalle_ventas = dca.final_sales_details()

# Escritura del DF final con el detalle de ventas

In [ ]:
wr = writer.Writer()
wr.processed_df(df_detalle_ventas,"processed/detalle_de_Ventas")

In [ ]:
df_final = spark.read.parquet("processed/detalle_de_Ventas")

In [ ]:
df_final.count()

In [ ]:
df_final.show(5, truncate = False)

# Clasificación de empresar de acuerdo a su desviación estandar

In [ ]:
df_performance = dca.company_clasification_by_stddev()

### Escritura

In [ ]:
wr.processed_df(df_performance, "processed/performance_por_compañia")

In [ ]:
df_performance.show(20,truncate=False)